Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

КРИТЕРИИ ОЦЕНИВАНИЯ:

* 1 балл	Верно выделены три столбца-признака для обучения, выбранные RFE.
* 1 балл	Верно выделены три столбца-признака для обучения, выбранные SelectKBest.
* 3 балла	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
* 3 балла	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
* 2 балла	Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

Максимальное количество баллов за выполнение задания — 10.

In [33]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [34]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [35]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

# Выделим три столбца-признака для обучения, выбранные RFE.

In [36]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

# Выделим три столбца-признака для обучения, выбранные SelectKBest.

In [37]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

# Обучим регрессия на первых трёх столбцах, оцененим качество модели на тесте.

In [38]:
data.dropna(inplace=True)

y = data['price']
X = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [42]:
X_train_rfe = X_train[['year', 'cylinders', 'lat']]
X_test_rfe = X_test[['year', 'cylinders', 'lat']]

model = LinearRegression()
model.fit(X_train_rfe, y_train)
y_predict = model.predict(X_test_rfe)

display(f'Test MAE: {metrics.mean_absolute_error(y_test, y_predict):.0f}')
display(f'Test MAPE: {metrics.mean_absolute_percentage_error(y_test, y_predict)*100:.0f}')
display(f'Test R^2: {metrics.r2_score(y_test, y_predict):.3f}')

'Test MAE: 5280'

'Test MAPE: 15314'

'Test R^2: 0.562'

# Обучим регрессию на вторых трёх столбцах, оценено качество модели на тесте.

In [40]:
X_train_skb = X_train[['year', 'cylinders', 'odometer']]
X_test_skb = X_test[['year', 'cylinders', 'odometer']]

model = LinearRegression()
model.fit(X_train_skb, y_train)
y_predict = model.predict(X_test_skb)


display(f'Test MAE: {metrics.mean_absolute_error(y_test, y_predict):.0f}')
display(f'Test MAPE: {metrics.mean_absolute_percentage_error(y_test, y_predict)*100:.0f}')
display(f'Test R^2: {metrics.r2_score(y_test, y_predict):.3f}')

'Test MAE: 4925'

'Test MAPE: 16291'

'Test R^2: 0.598'

# Вывод

Вывод: 
MAE - показывает насколько в среднем модель ошибается,чем ближе к нулю, тем лучше, MAE ниже у второй модели.

MAPE - показывает, на сколько процентов в среднем наше предсказание отклоняется от реального значения. чем ниже, тем лучше, выражается в %, ниже у первого.

R2 - показывает, насколько наша модель лучше, чем если бы все предсказания были средним по правильным ответам. Обе модели примерно равны, метрика ~0.6, модели можно считать удовлетворительными.

Судя по MAPE предсказания тестовой выборки отличпаются от реальных более чем на 15000 %, метрика для данного случия не верная? Или ошибка в том, что данные сырые?